In [4]:
!pip install XGBoost
!pip install catboost
!pip install optuna
!pip install LightGBM
!pip install optuna-integration[lightgbm]
import pandas as pd
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier


In [5]:
from sklearn.model_selection import train_test_split
df_m_train = pd.read_csv("train_Mortgage.csv")
df_p_train = pd.read_csv("train_Pension.csv")
df_s_train = pd.read_csv("train_Savings.csv")
target_name_1 = "Mortgage"
target_name_2 = "Pension"
target_name_3 = "Savings"
y_train_m = df_m_train[target_name_1]
y_train_p = df_p_train[target_name_2]
y_train_s = df_s_train[target_name_3]
X_train_m = df_m_train.drop(columns=[target_name_1])
X_train_p = df_p_train.drop(columns=[target_name_2])
X_train_s = df_s_train.drop(columns=[target_name_3])

In [6]:
df_m_valid = pd.read_csv("val_Mortgage.csv")
df_p_valid = pd.read_csv("val_Pension.csv")
df_s_valid = pd.read_csv("val_Savings.csv")
y_test_m = df_m_valid[target_name_1]
y_test_p = df_p_valid[target_name_2]
y_test_s = df_s_valid[target_name_3]
X_test_m = df_m_valid.drop(columns=[target_name_1])
X_test_p = df_p_valid.drop(columns=[target_name_2])
X_test_s = df_s_valid.drop(columns=[target_name_3])

CatBoost + ExtraTrees + LightGBM + XGBoost - > LogisticRegression

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Базовые модели
models = {
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=100, random_state=42),
    "LightGBM": LGBMClassifier(verbose=-1,random_state=42),
    "XGBoost": XGBClassifier(random_state=42)
}

def evaluate_stacking(X_train, X_test, y_train, y_test, models, meta_model):
    # Создаем стекинг
    stacking_model = StackingClassifier(
        estimators=list(models.items()),
        final_estimator=meta_model,
        cv=5,
        stack_method="predict_proba"
    )

    # Обучение и предсказание
    stacking_model.fit(X_train, y_train)
    y_pred = stacking_model.predict(X_test)
    y_proba = stacking_model.predict_proba(X_test)[:, 1]  # для ROC-AUC

    # Оценка
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    return accuracy, roc_auc

# Мета-модель
meta_model = LogisticRegression()

# Проверяем на каждом таргете
targets = {
    "Mortgage": (X_train_m, X_test_m, y_train_m, y_test_m),
    "Pension": (X_train_p, X_test_p, y_train_p, y_test_p),
    "Savings": (X_train_s, X_test_s, y_train_s, y_test_s)
}

results = {}
for target_name, (X_train, X_test, y_train, y_test) in targets.items():
    acc, auc = evaluate_stacking(X_train, X_test, y_train, y_test, models, meta_model)
    results[target_name] = {"Accuracy": acc, "ROC-AUC": auc}

# Вывод результатов
for target, metrics in results.items():
    print(f"{target}: Accuracy = {metrics['Accuracy']:.4f}, ROC-AUC = {metrics['ROC-AUC']:.4f}")

Mortgage: Accuracy = 0.7598, ROC-AUC = 0.9395
Pension: Accuracy = 0.5540, ROC-AUC = 0.7569
Savings: Accuracy = 0.5521, ROC-AUC = 0.6725


In [20]:
import pandas as pd
from itertools import combinations
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict

# Все модели для комбинирования

cb_mortgage = CatBoostClassifier(iterations=839, learning_rate=0.05774795442279825, depth=6,
                                 l2_leaf_reg=0.2880865175456189, random_strength=0.16254277485904814,
                                 bagging_temperature=0.7992786757360438, border_count=32, boosting_type='Ordered', verbose=0)
xgb_mortgage = XGBClassifier(n_estimators=234, max_depth=14, learning_rate=0.13020051661181956,
                                 subsample=0.5000183482043092, colsample_bytree=0.9712108470499956,
                                 min_child_weight=10, gamma=4.927111346239827, reg_alpha=2.3159564553387995,
                                 reg_lambda=1.7788240357504277, scale_pos_weight=1.155068929743032)

lgb_mortgage = LGBMClassifier(n_estimators=891, learning_rate=0.2775178145806826, num_leaves=154, max_depth=4,
                                  min_child_samples=5, subsample=0.875834146327149, colsample_bytree=0.9367342318343456,
                                  reg_alpha=4.0997572412343555, reg_lambda=1.3718029339702882, verbose=-1)

et_mortgage = ExtraTreesClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=2, n_estimators=200)
lr_mortgage = LogisticRegression(C=0.01, max_iter=1000, solver='lbfgs')


#mlp_mortgage = MLPClassifier(max_iter=300)


cb_pension = CatBoostClassifier(iterations=200, learning_rate=0.010875325239675968, l2_leaf_reg=0.010817134192675052,
                                random_strength=4.740509717654505, bagging_temperature=0.9584104224319896,
                                border_count=75, colsample_bylevel=0.25113614025415143, depth=1, boosting_type='Plain', verbose=0)
xgb_pension = XGBClassifier(n_estimators=238, max_depth=3, learning_rate=0.058427369161116946,
                                subsample=0.9293867219852593, colsample_bytree=0.5238187849054002,
                                min_child_weight=6, gamma=4.7308173402905505, reg_alpha=8.066467399907712,
                                reg_lambda=4.390801165747763, scale_pos_weight=1.0066125076750385)
lgb_pension = LGBMClassifier(n_estimators=1191, learning_rate=0.11153678049934727, num_leaves=107, max_depth=3,
                                 min_child_samples=41, subsample=0.5173705475484477, colsample_bytree=0.7026966245620551,
                                 reg_alpha=0.45638892145814025, reg_lambda=5.315359029396548, verbose=-1)
et_pension = ExtraTreesClassifier(max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=100)
lr_pension = LogisticRegression(C=0.001, max_iter=1000, solver='lbfgs')

cb_savings = CatBoostClassifier(iterations=205, learning_rate=0.041965496183629054, l2_leaf_reg=0.0033091053333706402,
                                random_strength=0.4267643014670958, bagging_temperature=0.6548287821971729, border_count=254,
                                colsample_bylevel=0.6355211623405239, depth=5, boosting_type='Plain', verbose=0)
xgb_savings = XGBClassifier(n_estimators=1446, max_depth=3, learning_rate=0.17633766607373635,
                                subsample=0.8802608079409611, colsample_bytree=0.6399123535237496,
                                min_child_weight=10, gamma=4.7135127571247475, reg_alpha=7.307229547015628,
                                reg_lambda=0.5506026977366328, scale_pos_weight=4.0245802517341716)
lgb_savings = LGBMClassifier(n_estimators=903, learning_rate=0.2620735099496511, num_leaves=198, max_depth=5,
                                 min_child_samples=98, subsample=0.5818744149073714, colsample_bytree=0.9979504029037664,
                                 reg_alpha=3.2759181752443514, reg_lambda=9.427794042824686, verbose=-1)

et_savings = ExtraTreesClassifier(max_depth=10, min_samples_leaf=1, min_samples_split=10, n_estimators=200)
lr_savings = LogisticRegression(C=0.1, max_iter=1000, solver='lbfgs')

all_models_m = {
    "CatBoost": cb_mortgage,
    "LightGBM": lgb_mortgage,
    "XGBoost": xgb_mortgage,
    "ExtraTrees": et_mortgage,
    "LogisticRegression" : lr_mortgage
}

all_models_p = {
    "CatBoost": cb_pension,
    "LightGBM": lgb_pension,
    "XGBoost": xgb_pension,
    "ExtraTrees": et_pension,
    "LogisticRegression" : lr_pension
}

all_models_s = {
    "CatBoost": cb_savings,
    "LightGBM": lgb_savings,
    "XGBoost": xgb_savings,
    "ExtraTrees": et_savings,
    "LogisticRegression" : lr_savings
}

# Мета-модель
meta_model = LogisticRegression()

# Перебираем все комбинации моделей от 2 до 6
min_comb_size = 2
max_comb_size = len(all_models_m)

results = {}


In [21]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [22]:
results = {}
def combs(min_comb_size, max_comb_size, all_models, X_train, y_train, X_test, y_test ):
    target_results=[]
    for r in range(min_comb_size, max_comb_size + 1):
        for model_names in combinations(all_models.keys(), r):
            models = {name: all_models[name] for name in model_names}

            stacking = StackingClassifier(
                estimators=list(models.items()),
                final_estimator=meta_model,
                cv=5,
                stack_method='predict_proba'
            )

            stacking.fit(X_train, y_train)
            y_proba = stacking.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_proba)

            target_results.append({
                "Models": "+".join(model_names),
                "ROC-AUC": auc,
                "Num_Models": r
            })
            print(target_results[-1])
    results[target_name] = pd.DataFrame(target_results).sort_values("ROC-AUC", ascending=False)
print("Mortgage")
combs(min_comb_size, max_comb_size, all_models_m, X_train_m, y_train_m, X_test_m, y_test_m)

for target_name, df in results.items():
    print(f"\n🔹 Лучшие комбинации для {target_name}:")
    print(df.head(10).to_markdown(tablefmt="grid"))

Mortgage
{'Models': 'CatBoost+LightGBM', 'ROC-AUC': np.float64(0.9421809196583971), 'Num_Models': 2}
{'Models': 'CatBoost+XGBoost', 'ROC-AUC': np.float64(0.9420636853069285), 'Num_Models': 2}
{'Models': 'CatBoost+ExtraTrees', 'ROC-AUC': np.float64(0.9402690979267556), 'Num_Models': 2}
{'Models': 'CatBoost+LogisticRegression', 'ROC-AUC': np.float64(0.9404043683322962), 'Num_Models': 2}
{'Models': 'LightGBM+XGBoost', 'ROC-AUC': np.float64(0.9425236046857668), 'Num_Models': 2}
{'Models': 'LightGBM+ExtraTrees', 'ROC-AUC': np.float64(0.9434975516056596), 'Num_Models': 2}
{'Models': 'LightGBM+LogisticRegression', 'ROC-AUC': np.float64(0.9431819206593982), 'Num_Models': 2}
{'Models': 'XGBoost+ExtraTrees', 'ROC-AUC': np.float64(0.9420456492528564), 'Num_Models': 2}
{'Models': 'XGBoost+LogisticRegression', 'ROC-AUC': np.float64(0.9420456492528564), 'Num_Models': 2}
{'Models': 'ExtraTrees+LogisticRegression', 'ROC-AUC': np.float64(0.939971503034566), 'Num_Models': 2}
{'Models': 'CatBoost+LightGB

In [23]:
print("Pension")
target_name = "Pension"
combs(min_comb_size, max_comb_size, all_models_p,X_train_p, y_train_p, X_test_p, y_test_p)

for target_name, df in results.items():
    print(f"\n🔹 Лучшие комбинации для {target_name}:")
    print(df.head(10).to_markdown(tablefmt="grid"))

Pension
{'Models': 'CatBoost+LightGBM', 'ROC-AUC': np.float64(0.7776058669078028), 'Num_Models': 2}
{'Models': 'CatBoost+XGBoost', 'ROC-AUC': np.float64(0.7781492527407801), 'Num_Models': 2}
{'Models': 'CatBoost+ExtraTrees', 'ROC-AUC': np.float64(0.7786594547060945), 'Num_Models': 2}
{'Models': 'CatBoost+LogisticRegression', 'ROC-AUC': np.float64(0.7781077729062017), 'Num_Models': 2}
{'Models': 'LightGBM+XGBoost', 'ROC-AUC': np.float64(0.7783317640129251), 'Num_Models': 2}
{'Models': 'LightGBM+ExtraTrees', 'ROC-AUC': np.float64(0.7765066512914748), 'Num_Models': 2}
{'Models': 'LightGBM+LogisticRegression', 'ROC-AUC': np.float64(0.7709483534579665), 'Num_Models': 2}
{'Models': 'XGBoost+ExtraTrees', 'ROC-AUC': np.float64(0.7784313156159134), 'Num_Models': 2}
{'Models': 'XGBoost+LogisticRegression', 'ROC-AUC': np.float64(0.7781948805588164), 'Num_Models': 2}
{'Models': 'ExtraTrees+LogisticRegression', 'ROC-AUC': np.float64(0.7782197684595633), 'Num_Models': 2}
{'Models': 'CatBoost+LightGB

In [24]:
print("Savings")
target_name = "Savings"
combs(min_comb_size, max_comb_size, all_models_s,X_train_s, y_train_s, X_test_s, y_test_s)

for target_name, df in results.items():
    print(f"\n🔹 Лучшие комбинации для {target_name}:")
    print(df.head(10).to_markdown(tablefmt="grid"))

Savings
{'Models': 'CatBoost+LightGBM', 'ROC-AUC': np.float64(0.6859416687090788), 'Num_Models': 2}
{'Models': 'CatBoost+XGBoost', 'ROC-AUC': np.float64(0.6917775988235706), 'Num_Models': 2}
{'Models': 'CatBoost+ExtraTrees', 'ROC-AUC': np.float64(0.6891404362592346), 'Num_Models': 2}
{'Models': 'CatBoost+LogisticRegression', 'ROC-AUC': np.float64(0.6924652498161828), 'Num_Models': 2}
{'Models': 'LightGBM+XGBoost', 'ROC-AUC': np.float64(0.6922551731381954), 'Num_Models': 2}
{'Models': 'LightGBM+ExtraTrees', 'ROC-AUC': np.float64(0.689744056580652), 'Num_Models': 2}
{'Models': 'LightGBM+LogisticRegression', 'ROC-AUC': np.float64(0.6955757851615839), 'Num_Models': 2}
{'Models': 'XGBoost+ExtraTrees', 'ROC-AUC': np.float64(0.6930646686040405), 'Num_Models': 2}
{'Models': 'XGBoost+LogisticRegression', 'ROC-AUC': np.float64(0.6961275865690978), 'Num_Models': 2}
{'Models': 'ExtraTrees+LogisticRegression', 'ROC-AUC': np.float64(0.6951808410069676), 'Num_Models': 2}
{'Models': 'CatBoost+LightGBM

In [9]:
# Базовые модели
base_models_1 = {
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "LightGBM": LGBMClassifier(verbose=-1,random_state=42),
}

# Мета-модель
meta_model = LogisticRegression()

results = {}
for target_name, (X_train, X_test, y_train, y_test) in targets.items():
    acc, auc = evaluate_stacking(X_train, X_test, y_train, y_test, models, meta_model)
    results[target_name] = {"Accuracy": acc, "ROC-AUC": auc}

# Вывод результатов
for target, metrics in results.items():
    print(f"{target}: Accuracy = {metrics['Accuracy']:.4f}, ROC-AUC = {metrics['ROC-AUC']:.4f}")

Mortgage: Accuracy = 0.7598, ROC-AUC = 0.9395
Pension: Accuracy = 0.5540, ROC-AUC = 0.7569
Savings: Accuracy = 0.5521, ROC-AUC = 0.6725


In [12]:
# Базовые модели
base_models_1 = {
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "LightGBM": LGBMClassifier(verbose=-1,random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=100, random_state=42),
}

# Мета-модель
meta_model = LogisticRegression()

results = {}
for target_name, (X_train, X_test, y_train, y_test) in targets.items():
    acc, auc = evaluate_stacking(X_train, X_test, y_train, y_test, base_models_1, meta_model)
    results[target_name] = {"Accuracy": acc, "ROC-AUC": auc}

# Вывод результатов
for target, metrics in results.items():
    print(f"{target}: Accuracy = {metrics['Accuracy']:.4f}, ROC-AUC = {metrics['ROC-AUC']:.4f}")

Mortgage: Accuracy = 0.7553, ROC-AUC = 0.9405
Pension: Accuracy = 0.5479, ROC-AUC = 0.7594
Savings: Accuracy = 0.5562, ROC-AUC = 0.6727


In [26]:
def optimize(meta_model, X_train, y_train, X_valid, y_valid, n_trials=10):
    def objective(trial):
        # LightGBM параметры
        params_lgbm = {
            'objective': 'binary',
            'metric': 'auc',
            'boosting_type': 'gbdt',
            'n_estimators': trial.suggest_int('lgbm_n_estimators', 50, 1400),
            'learning_rate': trial.suggest_float('lgbm_learning_rate', 0.01, 0.3, log=True),
            'num_leaves': trial.suggest_int('lgbm_num_leaves', 20, 300),
            'max_depth': trial.suggest_int('lgbm_max_depth', 3, 12),
            'min_child_samples': trial.suggest_int('lgbm_min_child_samples', 5, 100),
            'subsample': trial.suggest_float('lgbm_subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('lgbm_colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('lgbm_reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('lgbm_reg_lambda', 0.0, 10.0),
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': -1,
        }

        # CatBoost параметры
        params_catboost = {
            'iterations': trial.suggest_int("cat_iterations", 200, 1400),
            'learning_rate': trial.suggest_float('cat_learning_rate', 0.01, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('cat_l2_leaf_reg', 1e-3, 10.0, log=True),
            'random_strength': trial.suggest_float('cat_random_strength', 1e-3, 10.0, log=True),
            'bagging_temperature': trial.suggest_float('cat_bagging_temperature', 0.0, 1.0),
            'border_count': trial.suggest_int('cat_border_count', 32, 255),
            "colsample_bylevel": trial.suggest_float("cat_colsample_bylevel", 0.2, 0.7),
            "max_depth": trial.suggest_int("cat_depth", 1, 8),
            "boosting_type": trial.suggest_categorical("cat_boosting_type", ["Ordered", "Plain"]),
            'verbose': False,
            'random_state': 42,
        }

        # XGBoost параметры
        params_xg = {
            'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 1500),
            'max_depth': trial.suggest_int('xgb_max_depth', 3, 15),
            'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('xgb_subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('xgb_colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('xgb_min_child_weight', 1, 10),
            'gamma': trial.suggest_float('xgb_gamma', 0, 5),
            'reg_alpha': trial.suggest_float('xgb_reg_alpha', 0, 10),
            'reg_lambda': trial.suggest_float('xgb_reg_lambda', 0, 10),
            'scale_pos_weight': trial.suggest_float('xgb_scale_pos_weight', 1, 10),
            'tree_method': 'hist',
            'random_state': 42,
        }

        models = {
            "CatBoost": CatBoostClassifier(**params_catboost),
            "LightGBM": LGBMClassifier(**params_lgbm),
            "XGBoost": XGBClassifier(**params_xg)
        }

        stacking_model = StackingClassifier(
            estimators=list(models.items()),
            final_estimator=meta_model,
            cv=5,
            stack_method="predict_proba"
        )

        stacking_model.fit(X_train, y_train)
        y_pred_proba = stacking_model.predict_proba(X_valid)[:, 1]
        return roc_auc_score(y_valid, y_pred_proba)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Группируем параметры по моделям
    best_params = {
        'LightGBM': {k.replace('lgbm_', ''): v for k, v in study.best_params.items() if k.startswith('lgbm_')},
        'CatBoost': {k.replace('cat_', ''): v for k, v in study.best_params.items() if k.startswith('cat_')},
        'XGBoost': {k.replace('xgb_', ''): v for k, v in study.best_params.items() if k.startswith('xgb_')}
    }

    return best_params

In [24]:
X_train_m, X_valid_m, y_train_m, y_valid_m = train_test_split(X_train_m, y_train_m, test_size = 0.15, stratify=y_train_m)
X_train_p, X_valid_p, y_train_p, y_valid_p = train_test_split(X_train_p, y_train_p, test_size = 0.15, stratify=y_train_p)
X_train_s, X_valid_s, y_train_s, y_valid_s = train_test_split(X_train_s, y_train_s, test_size = 0.15, stratify=y_train_s)

Pension

In [27]:
meta_model = LogisticRegression()
best_stacking = optimize(meta_model, X_train_p, y_train_p, X_valid_p, y_valid_p)
best_stacking

[I 2025-05-02 11:55:06,866] A new study created in memory with name: no-name-7be5c71d-8cc7-48bb-a4a2-3417d8d371bf
[I 2025-05-02 11:55:18,204] Trial 0 finished with value: 0.7576946219697868 and parameters: {'lgbm_n_estimators': 1162, 'lgbm_learning_rate': 0.09321379091914742, 'lgbm_num_leaves': 32, 'lgbm_max_depth': 6, 'lgbm_min_child_samples': 46, 'lgbm_subsample': 0.8250467001169176, 'lgbm_colsample_bytree': 0.5770543438286264, 'lgbm_reg_alpha': 3.2537975553189016, 'lgbm_reg_lambda': 6.071629707453933, 'cat_iterations': 362, 'cat_learning_rate': 0.2938470962712474, 'cat_l2_leaf_reg': 0.0019474337570670882, 'cat_random_strength': 4.30158799289341, 'cat_bagging_temperature': 0.18525968786043778, 'cat_border_count': 124, 'cat_colsample_bylevel': 0.4161400414294646, 'cat_depth': 3, 'cat_boosting_type': 'Plain', 'xgb_n_estimators': 472, 'xgb_max_depth': 10, 'xgb_learning_rate': 0.29309710680731865, 'xgb_subsample': 0.779642379588581, 'xgb_colsample_bytree': 0.8089896646818704, 'xgb_min_ch

{'LightGBM': {'n_estimators': 1296,
  'learning_rate': 0.04319837725449642,
  'num_leaves': 39,
  'max_depth': 3,
  'min_child_samples': 67,
  'subsample': 0.8730498956876069,
  'colsample_bytree': 0.7626074410659165,
  'reg_alpha': 7.980290491436025,
  'reg_lambda': 2.463821499173208},
 'CatBoost': {'iterations': 1147,
  'learning_rate': 0.061122125116367235,
  'l2_leaf_reg': 0.0024097843044114378,
  'random_strength': 0.007568909245176004,
  'bagging_temperature': 0.6577039469224645,
  'border_count': 233,
  'colsample_bylevel': 0.4872587829334633,
  'depth': 6,
  'boosting_type': 'Ordered'},
 'XGBoost': {'n_estimators': 854,
  'max_depth': 8,
  'learning_rate': 0.2356928416385785,
  'subsample': 0.6728840728750967,
  'colsample_bytree': 0.7542727076520067,
  'min_child_weight': 10,
  'gamma': 0.2156438571350766,
  'reg_alpha': 3.056312656481536,
  'reg_lambda': 4.798472201652376,
  'scale_pos_weight': 8.497685576273852}}

In [35]:
meta_model = LogisticRegression()
best_stacking_m = optimize(meta_model, X_train_m, y_train_m, X_valid_m, y_valid_m, n_trials=20)
best_stacking_m

[I 2025-05-02 12:14:22,049] A new study created in memory with name: no-name-4eb5b2a4-35ad-48fc-b0df-bc52c5462ed1
[I 2025-05-02 12:15:52,787] Trial 0 finished with value: 0.9394333230674469 and parameters: {'lgbm_n_estimators': 1182, 'lgbm_learning_rate': 0.01009249405004522, 'lgbm_num_leaves': 163, 'lgbm_max_depth': 3, 'lgbm_min_child_samples': 26, 'lgbm_subsample': 0.8550560960526209, 'lgbm_colsample_bytree': 0.6904067389115587, 'lgbm_reg_alpha': 3.726407758746589, 'lgbm_reg_lambda': 2.688843350907064, 'cat_iterations': 1180, 'cat_learning_rate': 0.21806610364234755, 'cat_l2_leaf_reg': 0.049756191045619506, 'cat_random_strength': 0.058119390982250274, 'cat_bagging_temperature': 0.2562049032052469, 'cat_border_count': 146, 'cat_colsample_bylevel': 0.5494319141638592, 'cat_depth': 7, 'cat_boosting_type': 'Ordered', 'xgb_n_estimators': 297, 'xgb_max_depth': 8, 'xgb_learning_rate': 0.09876590791894875, 'xgb_subsample': 0.7113620282505199, 'xgb_colsample_bytree': 0.6835592572800222, 'xgb_

{'LightGBM': {'n_estimators': 64,
  'learning_rate': 0.025614328763480097,
  'num_leaves': 24,
  'max_depth': 5,
  'min_child_samples': 63,
  'subsample': 0.9950317449971098,
  'colsample_bytree': 0.5240943434801618,
  'reg_alpha': 9.311041322011077,
  'reg_lambda': 7.451864793343657},
 'CatBoost': {'iterations': 523,
  'learning_rate': 0.028720001263819364,
  'l2_leaf_reg': 0.9540469644063011,
  'random_strength': 0.0010745744717630063,
  'bagging_temperature': 0.3253719825405426,
  'border_count': 207,
  'colsample_bylevel': 0.21792118588158604,
  'depth': 4,
  'boosting_type': 'Plain'},
 'XGBoost': {'n_estimators': 1448,
  'max_depth': 6,
  'learning_rate': 0.1955495147680429,
  'subsample': 0.9632214870733696,
  'colsample_bytree': 0.513351716968891,
  'min_child_weight': 2,
  'gamma': 3.473414224878816,
  'reg_alpha': 7.725826976498152,
  'reg_lambda': 4.250983001181872,
  'scale_pos_weight': 9.957964683015014}}

In [40]:
meta_model = LogisticRegression()
best_stacking_s = optimize(meta_model, X_train_s, y_train_s, X_valid_s, y_valid_s, n_trials=25)
best_stacking_s

[I 2025-05-02 12:43:13,245] A new study created in memory with name: no-name-69584c6a-a450-42b7-8c41-28ea75a41ff3
[I 2025-05-02 12:43:41,668] Trial 0 finished with value: 0.6565874549502866 and parameters: {'lgbm_n_estimators': 661, 'lgbm_learning_rate': 0.1271903105477975, 'lgbm_num_leaves': 86, 'lgbm_max_depth': 9, 'lgbm_min_child_samples': 97, 'lgbm_subsample': 0.5298626395180501, 'lgbm_colsample_bytree': 0.8448196373236678, 'lgbm_reg_alpha': 7.716474144783056, 'lgbm_reg_lambda': 2.743853990130558, 'cat_iterations': 236, 'cat_learning_rate': 0.04692300096872996, 'cat_l2_leaf_reg': 0.18489073795440075, 'cat_random_strength': 3.1567721311652766, 'cat_bagging_temperature': 0.6732344760994166, 'cat_border_count': 170, 'cat_colsample_bylevel': 0.6041508481920668, 'cat_depth': 8, 'cat_boosting_type': 'Ordered', 'xgb_n_estimators': 581, 'xgb_max_depth': 6, 'xgb_learning_rate': 0.2041135455167327, 'xgb_subsample': 0.7501834135017863, 'xgb_colsample_bytree': 0.5615040588329131, 'xgb_min_chil

{'LightGBM': {'n_estimators': 263,
  'learning_rate': 0.04585667123878882,
  'num_leaves': 226,
  'max_depth': 4,
  'min_child_samples': 71,
  'subsample': 0.9952982670508074,
  'colsample_bytree': 0.5529531598888722,
  'reg_alpha': 4.686052128017627,
  'reg_lambda': 4.862253965409624},
 'CatBoost': {'iterations': 1112,
  'learning_rate': 0.014537587328026721,
  'l2_leaf_reg': 0.0025009091354511537,
  'random_strength': 0.29436664136961005,
  'bagging_temperature': 0.792252005481927,
  'border_count': 197,
  'colsample_bylevel': 0.3173367475980136,
  'depth': 1,
  'boosting_type': 'Ordered'},
 'XGBoost': {'n_estimators': 630,
  'max_depth': 11,
  'learning_rate': 0.2345706113867803,
  'subsample': 0.8747081814864642,
  'colsample_bytree': 0.9285570360405078,
  'min_child_weight': 1,
  'gamma': 0.5790079352269712,
  'reg_alpha': 7.323443986702505,
  'reg_lambda': 1.1666683179668764,
  'scale_pos_weight': 9.012438795315182}}

In [ ]:
models_s = {
            "CatBoost": CatBoostClassifier(**best_stacking_s['CatBoost'],verbose=False),
            "LightGBM": LGBMClassifier(**best_stacking_s['LightGBM']),
            "XGBoost": XGBClassifier(**best_stacking_s['XGBoost'])
        }
stacking_model_s = StackingClassifier(
            estimators=list(models_s.items()),
            final_estimator=meta_model,
            cv=5,
            stack_method="predict_proba"
        )

stacking_model_s.fit(X_train_s, y_train_s)
y_pred_proba = stacking_model_s.predict_proba(X_test_s)[:, 1]
roc_auc_score(y_test_s, y_pred_proba)

In [38]:
models_m = {
            "CatBoost": CatBoostClassifier(**best_stacking_m['CatBoost'],verbose=False),
            "LightGBM": LGBMClassifier(**best_stacking_m['LightGBM']),
            "XGBoost": XGBClassifier(**best_stacking_m['XGBoost'])
        }
stacking_model_m = StackingClassifier(
            estimators=list(models_m.items()),
            final_estimator=meta_model,
            cv=5,
            stack_method="predict_proba"
        )

stacking_model_m.fit(X_train_m, y_train_m)
y_pred_proba = stacking_model_m.predict_proba(X_test_m)[:, 1]
roc_auc_score(y_test_m, y_pred_proba)

np.float64(0.9418562706850995)

In [32]:
models = {
            "CatBoost": CatBoostClassifier(**best_stacking['CatBoost'],verbose=False),
            "LightGBM": LGBMClassifier(**best_stacking['LightGBM']),
            "XGBoost": XGBClassifier(**best_stacking['XGBoost'],verbose=False)
        }

stacking_model = StackingClassifier(
            estimators=list(models.items()),
            final_estimator=meta_model,
            cv=5,
            stack_method="predict_proba"
        )

stacking_model.fit(X_train_p, y_train_p)
y_pred_proba = stacking_model.predict_proba(X_test_p)[:, 1]
roc_auc_score(y_test_p, y_pred_proba)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:11:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:12:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:12:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:12:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:12:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, Use

np.float64(0.7749055296767478)

In [10]:
import ast
def optimizer_mlp(X_train, y_train, X_valid, y_valid, n_trials=10):
  def objective(trial):
    hidden_layers_str = trial.suggest_categorical(
        'hidden_layer_sizes',
        ['(50,)', '(100,)', '(50, 50)', '(100, 50)']
    )
    hidden_layers = ast.literal_eval(hidden_layers_str)  # безопасное преобразование строки в кортеж

    params = {
        'hidden_layer_sizes': hidden_layers,
        'activation': trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic']),
        'solver': trial.suggest_categorical('solver', ['adam', 'sgd']),
        'alpha': trial.suggest_float('alpha', 1e-4, 1e-1, log=True),
        'learning_rate': trial.suggest_categorical('learning_rate', ['constant', 'adaptive']),
        'max_iter': trial.suggest_int('max_iter', 200, 500),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        'early_stopping': trial.suggest_categorical('early_stopping', [True, False]),
        'random_state': 42
    }

    model = MLPClassifier(**params)
    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_valid)[:, 1]
    y_pred = model.predict(X_valid)
    return roc_auc_score(y_valid, y_pred_proba)


  study = optuna.create_study(direction='maximize')
  study.optimize(objective, timeout=3*60)
  return study.best_params

In [9]:
X_train_m, X_valid_m, y_train_m, y_valid_m = train_test_split(X_train_m, y_train_m, test_size = 0.15, stratify=y_train_m)
X_train_p, X_valid_p, y_train_p, y_valid_p = train_test_split(X_train_p, y_train_p, test_size = 0.15, stratify=y_train_p)
X_train_s, X_valid_s, y_train_s, y_valid_s = train_test_split(X_train_s, y_train_s, test_size = 0.15, stratify=y_train_s)

In [12]:
from sklearn.neural_network import MLPClassifier
best_m_mlp = optimizer_mlp(X_train_m, y_train_m, X_valid_m, y_valid_m, n_trials = 30)
best_m_mlp

[I 2025-05-02 15:54:58,908] A new study created in memory with name: no-name-c4908bbd-ab83-4af7-84c6-08bb8f7df6c4
[I 2025-05-02 15:54:59,612] Trial 0 finished with value: 0.5 and parameters: {'hidden_layer_sizes': '(50,)', 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.00014960217767804432, 'learning_rate': 'constant', 'max_iter': 435, 'batch_size': 128, 'early_stopping': False}. Best is trial 0 with value: 0.5.
[I 2025-05-02 15:55:10,178] Trial 1 finished with value: 0.49775378450578806 and parameters: {'hidden_layer_sizes': '(100, 50)', 'activation': 'logistic', 'solver': 'sgd', 'alpha': 0.0018687376320091125, 'learning_rate': 'adaptive', 'max_iter': 292, 'batch_size': 128, 'early_stopping': True}. Best is trial 0 with value: 0.5.
[I 2025-05-02 15:55:12,619] Trial 2 finished with value: 0.4959928762243989 and parameters: {'hidden_layer_sizes': '(100,)', 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0004116579633335589, 'learning_rate': 'constant', 'max_iter': 485, 'batch_size'

{'hidden_layer_sizes': '(50,)',
 'activation': 'relu',
 'solver': 'adam',
 'alpha': 0.0010268993228271025,
 'learning_rate': 'adaptive',
 'max_iter': 309,
 'batch_size': 64,
 'early_stopping': False}

In [21]:
#best_m_mlp['hidden_layer_sizes'] =  ast.literal_eval(best_m_mlp['hidden_layer_sizes'] )
model = MLPClassifier(**best_m_mlp)
model.fit(X_train_m, y_train_m)
y_pred_proba = model.predict_proba(X_test_m)[:, 1]
y_pred = model.predict(X_test_m)
roc_auc_score(y_test_m, y_pred_proba)

np.float64(0.8669705741777816)

In [14]:
best_p_mlp = optimizer_mlp(X_train_p, y_train_p, X_valid_p, y_valid_p, n_trials = 30)
best_p_mlp

[I 2025-05-02 15:58:40,990] A new study created in memory with name: no-name-92bc9d05-d7f8-43c2-b2cd-f42bc1ce02fc
[I 2025-05-02 15:58:41,900] Trial 0 finished with value: 0.5067813097671965 and parameters: {'hidden_layer_sizes': '(50,)', 'activation': 'logistic', 'solver': 'adam', 'alpha': 0.0001366839953164711, 'learning_rate': 'adaptive', 'max_iter': 264, 'batch_size': 64, 'early_stopping': False}. Best is trial 0 with value: 0.5067813097671965.
[I 2025-05-02 15:58:44,253] Trial 1 finished with value: 0.5 and parameters: {'hidden_layer_sizes': '(50,)', 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.051816140493377724, 'learning_rate': 'adaptive', 'max_iter': 427, 'batch_size': 64, 'early_stopping': False}. Best is trial 0 with value: 0.5067813097671965.
[I 2025-05-02 15:59:03,804] Trial 2 finished with value: 0.47730000659500094 and parameters: {'hidden_layer_sizes': '(100, 50)', 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0005093691065803074, 'learning_rate': 'adaptive', 'm

{'hidden_layer_sizes': '(100,)',
 'activation': 'relu',
 'solver': 'adam',
 'alpha': 0.00015474065316415945,
 'learning_rate': 'constant',
 'max_iter': 448,
 'batch_size': 32,
 'early_stopping': False}

In [22]:
best_p_mlp['hidden_layer_sizes'] =  ast.literal_eval(best_p_mlp['hidden_layer_sizes'] )
model = MLPClassifier(**best_p_mlp)
model.fit(X_train_p, y_train_p)

y_pred_proba = model.predict_proba(X_test_p)[:, 1]
y_pred = model.predict(X_test_p)
roc_auc_score(y_test_p, y_pred_proba)

np.float64(0.7589627552565321)

In [15]:
best_s_mlp = optimizer_mlp(X_train_s, y_train_s, X_valid_s, y_valid_s, n_trials = 30)
best_s_mlp

[I 2025-05-02 16:01:49,640] A new study created in memory with name: no-name-933d37d0-5414-4c57-81ae-ea7d5c122b02
[I 2025-05-02 16:01:53,905] Trial 0 finished with value: 0.5014623699578451 and parameters: {'hidden_layer_sizes': '(100, 50)', 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.03493516146980141, 'learning_rate': 'constant', 'max_iter': 292, 'batch_size': 128, 'early_stopping': False}. Best is trial 0 with value: 0.5014623699578451.
[I 2025-05-02 16:01:58,736] Trial 1 finished with value: 0.5491525505665777 and parameters: {'hidden_layer_sizes': '(100,)', 'activation': 'logistic', 'solver': 'sgd', 'alpha': 0.004761953179348651, 'learning_rate': 'adaptive', 'max_iter': 263, 'batch_size': 128, 'early_stopping': True}. Best is trial 1 with value: 0.5491525505665777.
[I 2025-05-02 16:02:00,378] Trial 2 finished with value: 0.38291855203619896 and parameters: {'hidden_layer_sizes': '(100, 50)', 'activation': 'logistic', 'solver': 'adam', 'alpha': 0.000623230670392599, 'learning

{'hidden_layer_sizes': '(100,)',
 'activation': 'relu',
 'solver': 'adam',
 'alpha': 0.018600215630507698,
 'learning_rate': 'constant',
 'max_iter': 274,
 'batch_size': 64,
 'early_stopping': False}

In [23]:
best_s_mlp['hidden_layer_sizes'] =  ast.literal_eval(best_s_mlp['hidden_layer_sizes'] )
model = MLPClassifier(**best_s_mlp)
model.fit(X_train_s, y_train_s)

y_pred_proba = model.predict_proba(X_test_s)[:, 1]
y_pred = model.predict(X_test_s)
roc_auc_score(y_test_s, y_pred_proba)

np.float64(0.6929302195301286)